# Learning to use MongoDB

_2020-06-20_

* Jose RazGuzman
---

## Initialization

In [4]:
import pymongo
from pymongo import MongoClient
import json
import pprint
import os
import pandas as pd
import numpy as np
client = MongoClient("mongodb://mongo:27017")
db = client.db
# Connect
if("db" not in client.list_database_names() \
   or "zipcodes" not in db.list_collection_names() \
   or "prescriptions" not in db.list_collection_names()):
    os.system('python2 HW2-import.py')

# Checking if the data is correctly inserted:
[db.zipcodes.count_documents({}), db.prescriptions.count_documents({})]

[29353, 60076]

# Queries for "Zip codes"

## Q1
Count the total number of cities in Washington state (code: "WA")

In [553]:
# First we match the citites that correspond to the state of Washington by using the code "WA"
# Then we use two group functions: the first groups together the cities from the states to avoid duplicates 
# The second group counts the total total number of cities 
q1 = db.zipcodes.aggregate([
    { "$match": 
         { "state": "WA" } },
    { "$group": {
            "_id": "$city" } },
    { "$group": {
            "_id": "Washington",
            "count": { "$sum": 1 } } }
]);
q1_print = pd.DataFrame(list(q1)).rename(columns={'_id': 'State', 'count': 'Number of cities'})
q1_print

,State,Number of cities
0,Washington,397


## Q2
Find the total population of each state (i.e., sort states by their population in the ascending order)

In [554]:
# We take the aggregate of each state by using _id: "state"
# We define and group the new attribute "total_pop" as the sum of the population by state
# Sort "total_pop" using $sort and specifying ascending order by declaring the value of 1
# To output the results in proper format we convert to a pandas dataframe
# We change the names of the columns to reflect the appropiate values and reindex starting from 1
# Print the list of state populations to the console
q2 = db.zipcodes.aggregate(
    [
        {"$group": {"_id": "$state", "total_pop": {"$sum": "$pop"} } },
        { "$sort": { "total_pop" : 1}}
    ]
)
q2_print = pd.DataFrame((list(q2)))
q2_print = q2_print.rename(columns={'_id': 'State', 'total_pop': 'Total Population'})
q2_print.index = np.arange(1, len(q2_print) + 1)
q2_print

,State,Total Population
1,WY,453528
2,AK,544698
3,VT,562758
4,DC,606900
5,ND,638272
6,DE,666168
7,SD,695397
8,MT,798948
9,RI,1003218
10,ID,1006749


## Q3
Find the 10 closest cities to WEST BROOKLYN, IL. You might want to use the $near operator

In [443]:
# The first step is to print out the information regarding WEST BROOKLYN, IL to obtain the longitude and latitude of the city.
pprint.pprint(list(db.zipcodes.find({"state":"IL", "city": "WEST BROOKLYN"})))

[{u'_id': u'61378',
  u'city': u'WEST BROOKLYN',
  u'loc': [-89.190917, 41.729156],
  u'pop': 946,
  u'state': u'IL'}]


In [444]:
# To use the $near operator we have to create a special index for pymongo to recognize the coordinates locations
# We specify that coordinates are for a 2dsphere geolocation
db.zipcodes.create_index( [( "loc" , "2dsphere" )] )
#db.zipcodes.create_index( [( "point" , "2dsphere" )] )
#db.zipcodes.index_information()

u'point_2dsphere'

In [555]:
# Query 3 uses the function "find" on the first parameter of "loc"
# We define the special parameter $near and specify the type of "geometry" that will be passed on
# The two parameter necessary are "type", which is assigned "point", 
# and "coordinates" which are the ones taken from WEST BROOKLYN, IL 
# We limit our find to the first eleven cities because the closest city will be itself
q3 = db.zipcodes.find({
    "loc": {
        "$near": {
            "$geometry": { 
                "type": "Point", 
                "coordinates": [-89.190917, 41.729156] 
            }
        }
    }}).limit(11)

# To output the results in proper format we convert to a pandas dataframe
# We change the names of the columns to reflect the appropiate values and print out the cities starting from index 1 onwards
q3_print = pd.DataFrame((list(q3))).rename(columns={'loc': 'location', 'pop': 'population'})
q3_print.iloc[1:,:]

,_id,city,location,population,state
1,61318,COMPTON,"[-89.087708, 41.684976]",551,IL
2,61367,SUBLETTE,"[-89.235409, 41.633144]",899,IL
3,61310,AMBOY,"[-89.34716, 41.704181]",3994,IL
4,61006,ASHTON,"[-89.2086, 41.864327]",1911,IL
5,61031,FRANKLIN GROVE,"[-89.317112, 41.857968]",2070,IL
6,61353,PAW PAW,"[-88.967377, 41.685228]",1539,IL
7,60530,LEE,"[-88.971386, 41.786418]",825,IL
8,60553,STEWARD,"[-89.015086, 41.847545]",661,IL
9,61342,MENDOTA,"[-89.10828, 41.544308]",9660,IL
10,61330,LA MOILLE,"[-89.297024, 41.537557]",1315,IL


## Q4
Considering the `region` of each US state, according to this source (below), find the total population in each of the four regions (West, South, Midwest, and Northeast)

https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States  

https://pe.usps.com/text/pub28/28apb.htm

Region 1: Northeast
* Division 1: New England (Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, and Vermont)
* Division 2: Mid-Atlantic (New Jersey, New York, and Pennsylvania)

Region 2: Midwest
* Division 3: East North Central (Illinois, Indiana, Michigan, Ohio, and Wisconsin)
* Division 4: West North Central (Iowa, Kansas, Minnesota, Missouri, Nebraska, North Dakota, and South Dakota)

Region 3: South
* Division 5: South Atlantic (Delaware, Florida, Georgia, Maryland, North Carolina, South Carolina, Virginia, District of Columbia, and West Virginia)
* Division 6: East South Central (Alabama, Kentucky, Mississippi, and Tennessee)
* Division 7: West South Central (Arkansas, Louisiana, Oklahoma, and Texas)

Region 4: West
* Division 8: Mountain (Arizona, Colorado, Idaho, Montana, Nevada, New Mexico, Utah, and Wyoming)
* Division 9: Pacific (Alaska, California, Hawaii, Oregon, and Washington)

In [556]:
# First we create four lists with the codes for each state in the requested regions
northeast = ["CT", "ME", "MA", "NH", "RI", "VT", "NJ", "NY", "PA"]
midwest = ["IL", "IN", "MI", "OH", "WI", "IA", "KS", "MN", "MO", "NE", "ND", "SD"]
south = ["DE", "FL", "GA", "MD", "NC", "SC", "VA", "DC", "WV", "AL", "KY", "MS", "TN", "AR", "LA", "OK", "TX"]
west = ["AZ", "CO", "ID", "MT", "NV", "NM", "UT", "WY", "AK", "CA", "HI", "OR", "WA"]
# Checking I have the all the states necessary
print(len(northeast+midwest+south+west) == len(db.zipcodes.distinct('state')))

True


In [557]:
# Next we use an aggregate function to aggregate the values for each region
# The instruction are to first match the state code using "$in" with the respective list of each region
# Then to group the values of the population by adding to obtain the total population per region
# We repeat the steps for each of the four regions
northeast_pop = db.zipcodes.aggregate([
    { "$match": 
         { "state": {"$in": northeast} } },
    {"$group": {
        "_id": "Northeast", 
        "total_pop": {"$sum": "$pop"} } }
])
midwest_pop = db.zipcodes.aggregate([
    { "$match": 
         { "state": {"$in": midwest} } },
    {"$group": {
        "_id": "Midwest", 
        "total_pop": {"$sum": "$pop"} } }
])
south_pop = db.zipcodes.aggregate([
    { "$match": 
         { "state": {"$in": south} } },
    {"$group": {
        "_id": "South", 
        "total_pop": {"$sum": "$pop"} } }
])
west_pop = db.zipcodes.aggregate([
    { "$match": 
         { "state": {"$in": west} } },
    {"$group": {
        "_id": "West", 
        "total_pop": {"$sum": "$pop"} } }
])

# To output the results in proper format we add the results of the four regions and convert to a pandas dataframe
# We change the names of the columns and index to reflect the appropiate values and print out
region_pop = list(northeast_pop)+list(midwest_pop)+list(south_pop)+list(west_pop)
q4_print = pd.DataFrame(region_pop).rename(columns={'_id': 'Region', 'total_pop': 'Total Population'})
q4_print.index = np.arange(1, len(q4_print) + 1)
q4_print

,Region,Total Population
1,Northeast,50807650
2,Midwest,59652438
3,South,85173522
4,West,52774790


## Q5
Find the 3 most populated cities for each state (in one query)

In [5]:
# First we group by state and city, this allows us to make the sum of city populations
# Next we sort all the cities by populations in descdending order
# We follow by grouping again, on this instance we use the push operator to append the city populations
# Then we use project and the key aspect is that we use the operator "slice" to obtain the first 3 cities of each state
# We proceed to use "unwind" to return an array for each individual city
# We project this array to extract the nested values so its easier to read when it prints out
q5= db.zipcodes.aggregate([
    { "$group":{
       "_id": { 
           "state": "$state", 
           "city": "$city" },
       "city_pop": { "$sum": "$pop" } } },
    { "$sort": { "city_pop": -1 } },
    { "$group":{
       "_id": "$_id.state",
       "city": {
           "$push": {
               "city":"$_id.city",
               "city_pop": "$city_pop" } } } },
    { "$project": {
       "cities": { "$slice": [ "$city", 3 ] },
       "city_pop": "$city_pop" } },
    { "$unwind": "$cities"},
    { "$project": { 
        "_id": 1, 
        "city": "$cities.city",
        "city_pop": "$cities.city_pop"} }
])

# To output the results in proper format we convert to a pandas dataframe
# We change the names of the columns and index to reflect the appropiate values and print out
q5_print = pd.DataFrame((list(q5))).rename(columns={'_id': 'State', 'city': 'City', 'city_pop': 'City Population'})
q5_print

,State,City,City Population
0,IA,DES MOINES,148155
1,IA,CEDAR RAPIDS,116146
2,IA,DAVENPORT,95743
3,IL,CHICAGO,2452177
4,IL,ROCKFORD,167031
5,IL,LINCOLNWOOD,111365
6,AR,LITTLE ROCK,192895
7,AR,FORT SMITH,76584
8,AR,JONESBORO,53532
9,FL,MIAMI,825232


## Q6
Find the specialty of all doctors who have prescribed "HALOPERIDOL".

In [6]:
# We begin by first using unwind to unpack the specialty of each doctor in 'provider_variables.specialty'
# We project that we want to return the doctor id and the specialty
# Specialty has a conditional statement, we are looking for doctor who have prescribed HALOPERIDOL
# We use an if statement that check whether the prescription counter for the drug is greater than or equal to one
# Else we return a None value
# We match the Specialties projected that are not equal to None to get rid of doctors that have not prescribed the drug
q6 = db.prescriptions.aggregate([
    {"$unwind":"$provider_variables.specialty"},
    {"$project": {
        "_id": "$npi", 
        "Specialty":{
            "$cond": { 
                "if": { "$gte": ["$cms_prescription_counts.HALOPERIDOL", 1] }, 
                "then": "$provider_variables.specialty", 
                "else": None }
                }
            }},
    {"$match": {
        "Specialty":{"$ne": None}
            }}
])

# To output the results in proper format we convert to a pandas dataframe
# We change the names of the columns and index to reflect the appropiate values and print out
q6_print = pd.DataFrame((list(q6))).rename(columns={'_id': 'Doctor NPI'})
q6_print.index = np.arange(1, len(q6_print) + 1)
q6_print

,Specialty,Doctor NPI
1,Psychiatry,1215979554
2,Psychiatry,1174564165
3,Psych/Mental Health,1912043050
4,Psych/Mental Health,1043561624
5,Psychiatry,1356420244
6,Psychiatry,1306880281
7,Psychiatry,1457566069
8,Psychiatry,1750359600
9,Psychiatry,1942281019
10,Psychiatry,1033218631


## Q7
Find the total number of doctors, separately for each region (in one query)

In [7]:
# Query 7 uses the aggregate function to get the sum of counts
# We define parameter $group and pass on the id of the nested variables $provider_variables.region
# Additionally we keep a counter and sum each instance under the assumption there are no duplicate doctor values
q7 = db.prescriptions.aggregate([
    {"$group": 
         {"_id": "$provider_variables.region", 
          "count": {"$sum": 1} 
         } 
    }
])

# To output the results in proper format we convert to a pandas dataframe
# We change the names of the columns to reflect the appropiate values and print out
q7_print = pd.DataFrame((list(q7))).rename(columns={'_id': 'Region', 'count': 'Number of Doctors'})
q7_print.index = np.arange(1, len(q7_print) + 1)
q7_print

,Region,Number of Doctors
1,South,20017
2,Northeast,14801
3,West,12657
4,Midwest,12601


## Q8
Find the total amount of prescribed "ATORVASTATIN CALCIUM"

In [8]:
# We assume that the value of each drug is the “number of times it is prescribed”
# We begin by first using unwind to unpack the prescriptions counts from 'cms_prescription_counts'
# We group together as a $sum the medicine specified and declare _id as 0, because we only care about total amount
# We project the results and declare _id = to avoid passing it forward, and 'total_amount' as 1 to have it show as the result
q8 = db.prescriptions.aggregate([
    {"$unwind": "$cms_prescription_counts"},
    {"$group": {
        "_id": 0, 
        "total_amount": {"$sum": "$cms_prescription_counts.ATORVASTATIN CALCIUM"}
        }},
    { "$project": { "_id": 0, "total_amount": 1 } },
])

# To output the results in proper format we convert to a pandas dataframe
# We change the names of the columns and the name of the index to reflect the appropiate values and print out
q8_print = pd.DataFrame((list(q8))).rename(columns={'total_amount': 'Total Amount Prescribed'})
q8_print.rename(index={0:'ATORVASTATIN CALCIUM'}, inplace=True)
q8_print

,Total Amount Prescribed
ATORVASTATIN CALCIUM,1143467


## Q9
Find the drug that is prescribed by the most of doctors working in "non-urban" areas. (in terms of number of doctors who prescribed it, not the total amount of prescriptions).

In [9]:
# First we match by "non-urban" areas which is found inside provider variables under "settlement type"
# Then we use project and object to Array on the cms prescriptions counts to flatten the list of medicines
# We follow with unwind which peel off a document for each medicine and returns that resulting document
# Now we group by the value of keys create in "drugs" using drugs.k and count the instances of each prescriptions
# Finally we sort the value of count in desceding order to the get most prescribed drug of the area 
# We only need the most prescribed drug so we use limit to get the first value
q9 = db.prescriptions.aggregate([
    { "$match": 
         { "provider_variables.settlement_type": "non-urban" } },
    { "$project": 
         { "drugs" : 
              { "$objectToArray": "$cms_prescription_counts" } } },
    { "$unwind": "$drugs"},
    { "$group": 
         {"_id": "$drugs.k", 
          "count": {"$sum": 1} } },
    { "$sort": 
         { "count": -1 } },
    { "$limit": 1}
])

# To output the results in proper format we convert to a pandas dataframe
# We change the names of the columns and the name of the index to reflect the appropiate values and print out
q9_print = pd.DataFrame(list(q9)).rename(columns={'_id': 'Drug', 'count': 'Number of prescriptions'})
q9_print.rename(index={0:'Most prescribed in "non-urban" areas'}, inplace=True)
q9_print

,Drug,Number of prescriptions
"Most prescribed in ""non-urban"" areas",HYDROCODONE-ACETAMINOPHEN,10776


## Q10
Considering the region of US states (Query #4 ) and the region of each doctor, find the average number of doctors per capita in each of the four regions in US

_NOTE_: The method used assumes the answer for average number of doctor per capita can be given as "Doctors per 10,000 people"

However, the loading of the second data contained a bug and a fewer number of rows are computed, therefore the answer may vary from the proper solution.

Results from Q4
    Region | Total Population
* 1	Northeast =	50807650
* 2	Midwest =	59652438
* 3	South =	85173522
* 4	West =	52774790

In [28]:
# Using the code from Q7 we group the region to obtain the count per region
# Then we project and include a "switch" clause that will check the for the id of each region 
# and divide the population according to the results of Q4
# We project again and multiply the divided number to obtain the doctors per capita in the format of "Doctors per 10,000 people"
# The format makes it much easier to compare the average number of doctors between regions
# NOTE the value is much lower than expected due to a reduced dataset
# Finally a project step to round the number of doctors for readability
q10 = db.prescriptions.aggregate([
    {"$group": {
            "_id": "$provider_variables.region", 
            "count": {"$sum": 1} } },
    { "$project": { 
         "_id": 1,
         "count": { 
             "$switch": {
                "branches": [
                    { "case": { "$eq": ["$_id", "Northeast"] }, 
                         "then": { "$divide": [ "$count", 50807650 ] } },
                    { "case": { "$eq": ["$_id", "Midwest"] }, 
                         "then": { "$divide": [ "$count", 59652438 ] } },
                    { "case": { "$eq": ["$_id", "South"] }, 
                         "then": { "$divide": [ "$count", 85173522 ] } },
                    { "case": { "$eq": ["$_id", "West"] }, 
                         "then": { "$divide": [ "$count", 52774790 ] } },
                ],
                "default": "1"
              } } } },
    { "$project": { 
         "_id": 1,
         "doc_per_capita": { 
             "$multiply": [ "$count", 10000 ] } } },
    { "$project": { 
        "doc_per_capita": { 
            "$round": [ "$doc_per_capita", 1 ] } } }
])
# To output the results in proper format we convert to a pandas dataframe
# We change the names of the columns and the name of the index to reflect the appropiate values and print out
q10_print = pd.DataFrame((list(q10))).rename(columns={'_id': 'Region', 'doc_per_capita': 'Doctors per 10,000 people'})
q10_print.index = np.arange(1, len(q10_print) + 1)
q10_print

,Region,"Doctors per 10,000 people"
1,South,2.4
2,Northeast,2.9
3,West,2.4
4,Midwest,2.1
